In [ ]:
import requests
import re
import os
import nltk
from bs4 import BeautifulSoup

from urllib.parse import urlsplit
from nltk.corpus import stopwords
from pyvi import ViTokenizer
from textblob import TextBlob
import shutil

from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.probability import FreqDist

import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from pyvi import ViTokenizer, ViPosTagger #pip install underthesea


## crawl

In [ ]:
url = 'https://plo.vn/phap-luat/'
file_store = "D:/22/Cau1/Data"
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')

contents = soup.find_all('article', class_='story')
data_crawl = []

for content in contents:
    if content.has_attr('data-id'):
        data_crawl.append(content)

In [ ]:

i=1
while i <= 10:
    Tieude = data_crawl[i].find('a', class_='cms-link')
    print(Tieude.text)
    Ngay = data_crawl[i].find('time', class_ = 'time')
    Tomtat = data_crawl[i].find('div', class_ = 'story__summary story__shorten')

    if Tieude or Ngay or Tomtat is not None:
        link = Tieude.get('href')
        title = Tieude['title']
        time = Ngay['datetime']
        desc = Tomtat.text

        print(link)
        print(title)
        print(time)
        print(desc)
        print("\n")

    filename=os.path.join(file_store, f"22_news{i}.txt")   
    with open(filename, 'w',encoding='utf-8') as f:
        f.write(f"{link}" + "\n")
        f.write(f"{title}" + "\n")
        f.write(f"{time}" + "\n")
        f.write(f"{desc}")
    i+=1


## Process

In [ ]:
# Thư mục nguồn
source_dir = 'D:/22/Cau2/Data'

# Thư mục đích
destination_dir = 'D:/22/Cau2/Data/Preprocessing'


# Hàm xóa các thẻ HTML không cần thiết
def XoaTag(html):
    soup = BeautifulSoup(html, 'html.parser')
    for data in soup(['style', 'script']):
        data.decompose()
    return ' '.join(soup.stripped_strings)

i=1

for file in os.listdir(source_dir):
    try:
        # Đọc nội dung của tệp
        with open(os.path.join(source_dir, file), 'r', encoding='utf-8') as f:
            text = f.read()

        text_pre=text.replace("\n","")    
        text_pre=text.lower() # Chuyển văn bản thành chữ thường    
        text_pre = re.sub(r"http(s)?://\S+", "", text_pre) # Loại bỏ URL
        text_pre = XoaTag(text_pre) # Xoá thẻ HTML
        text_pre=re.sub(r'[^\w\s]','',text_pre) # Xóa dấu câu
        text_pre = re.sub("\d+", " ", text_pre) # Xóa chữ số
        text_pre = re.sub(r"[!@#$[]()]'", "", text_pre) # Xoá kí tự đặc biệt: !@#$[]()
        text_pre = nltk.sent_tokenize(text_pre) # Tách câu
        text_pre = ''.join(text_pre)

        # Xóa các từ không có nghĩa (Stop Words);
        path=os.path.join("D:/22")
        f = open(path+r"/vietnamese-stopwords.txt", "r", encoding="utf-8")
        List_StopWords=f.read().split("\n")
        text_pre=" ".join(text_pre for text_pre in text_pre.split() if text_pre not in List_StopWords)

        text_pre = ViTokenizer.tokenize(text_pre) # Tách từ (Tokenizing)

        print (text_pre)

        # Ghi nội dung đã được tiền xử lý vào một tệp mới
        with open(os.path.join(destination_dir, f'22_Pre{i}.txt'), 'w',encoding='utf-8') as f:
            for word in text_pre:
                f.write(word)
        i+=1
    except:
        break

## Tan Suat Tu, WordCloud

In [ ]:
filestore = 'D:/22/Cau3/Data'

text_pre = ''
for file in os.listdir(filestore):
  # Đọc nội dung của tệp
    with open(os.path.join(filestore, file), 'r', encoding='utf-8') as f:
        text = f.read()
        text_pre += text

text_pre1=nltk.word_tokenize(text_pre) # Tokenizing

print("Number of words: ",len(text_pre1))

# Compute the frequency of all words / Tính tần số của tất cả các từ
frequency_dist = FreqDist(word.lower() for word in text_pre1)

## show only th top 50 results
print(frequency_dist.most_common(50))

## Consider words with length greater than 3 and plot
# Xem xét các từ có độ dài lớn hơn 3 và vẽ
large_words = dict([(k,v) for k,v in frequency_dist.items() if len(k)>3])
frequency_dist = nltk.FreqDist(large_words)
frequency_dist.plot(50,cumulative=False) # top 50 từ đầu

In [ ]:
# WORD CLOUD
# Convert the text to a dictionary of word frequencies / Chuyển đổi văn bản thành từ điển tần số từ
word_counts = nltk.FreqDist(text_pre.split())
# Generate the word cloud / Tạo đám mây từ
wcloud = WordCloud().generate_from_frequencies(frequency_dist)
#plotting the wordcloud
plt.imshow(wcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

## Do Tuong Dong

In [ ]:
def read_file(filename):
    filename = os.path.join(Path, filename)
    with open(filename, 'r', encoding="utf-8") as f:
        data = f.read()  # Đọc dữ liệu như một chuỗi

    return data  # Trả về chuỗi mà không mã hóa lại thành bytes


# Chuẫn hóa dữ liệu
def create_dataframe(matrix, tokens):
    doc_names = [f'doc_{i+1}' for i, _ in enumerate(matrix)]
    df = pd.DataFrame(data=matrix, index=doc_names, columns=tokens)
    return(df)

# # Tiền xử lý văn bản
# import re
# def Text_Preprocessing(doc):
#     text_pre = doc.lower()
#     text_pre=text_pre.replace("\n","")    
#     text_pre=text_pre.lower() # Chuyển văn bản thành chữ thường    
#     text_pre = re.sub(r"http(s)?://\S+", "", text_pre) # Loại bỏ URL
#     text_pre=re.sub(r'[^\w\s]','',text_pre) # Xóa dấu câu
#     text_pre = re.sub("\d+", " ", text_pre) # Xóa chữ số
#     text_pre = re.sub(r"[!@#$[]()]'", "", text_pre) # Xoá kí tự đặc biệt: !@#$[]()
#     text_pre = ''.join(text_pre)

#     # # Xóa các từ không có nghĩa (Stop Words);
#     # text_pre = ViTokenizer.tokenize(text_pre) # Tách từ (Tokenizing)
#         ##Remove stop words
#     filename=os.path.join(r"D:\Phuong\KhaiPhaDuLieuWeb\Chương 5\vietnamese-stopwords.txt")
#     f = open(filename, "r", encoding="utf-8")
    
#     # #Get Stop words Dictionaries
#     List_StopWords=f.read().split("/n")
#     text_pre=" ".join(text for text in text_pre.split() if text not in List_StopWords)
    
#     text_pre = ViTokenizer.tokenize(text_pre)

#     return text_pre

In [ ]:
# Thêm tất cả các từ trong tập văn bản vào một tập hợp
doc_ = []
for i in range(1,4):
    filestore = f'D:/Phuong/KhaiPhaDuLieuWeb/Chương 5/BTCh5/{i}'
    for file in os.listdir(filestore):
        text_pre =''
        with open(os.path.join(filestore, file), 'r', encoding='utf-16 LE') as f:
            text = f.read()
            text_pre += text
        doc_.append(text_pre)

doc_


In [ ]:
data = []
for doc in doc_:
    doc_pre=Text_Preprocessing(doc)
    data.append(doc_pre)

data = data

In [ ]:
# Xây dựng vector TF-IDF
# data = [' '.join(doc) for doc in data]
CountVect = CountVectorizer()
vector_matrix = CountVect.fit_transform(data)
vector_matrix.shape

In [ ]:
# Lấy tên các từ (features)
tokens = CountVect.get_feature_names_out()
# Tạo DataFrame từ vector_matrix
vector_df = create_dataframe(vector_matrix.toarray(), tokens)

# Tính toán độ tương đồng cosine giữa các tài liệu
cosine_similarity_matrix = cosine_similarity(vector_matrix)

# Tạo doc_names từ doc_1 đến doc_n
doc_names = [f'doc_{i+1}' for i in range(cosine_similarity_matrix.shape[0])]

# Tạo DataFrame từ ma trận cosine similarity
Similarity = create_dataframe(cosine_similarity_matrix, doc_names)

# In ra DataFrame tương đồng cosine
print(Similarity)

In [ ]:
import pandas as pd

# Tạo DataFrame từ ma trận cosine similarity
Similarity_df = pd.DataFrame(Similarity, index=doc_names, columns=doc_names)

# Chuyển DataFrame thành dạng long-form
similarity_long = Similarity_df.stack().reset_index()
similarity_long.columns = ['doc_1', 'doc_2', 'Do_tuong_dong']

# Loại bỏ các dòng có doc_1 == doc_2 (để tránh độ tương đồng của tài liệu với chính nó)
similarity_long = similarity_long[similarity_long['doc_1'] != similarity_long['doc_2']]

# Sắp xếp theo độ tương đồng tăng dần
similarity_sorted = similarity_long.sort_values(by='Do_tuong_dong', ascending=False)

# In ra kết quả
print(similarity_sorted)
